<a href="https://colab.research.google.com/github/t8101349/Colab-/blob/master/BigQuery%E5%9F%BA%E7%A4%8E%E7%AE%A1%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

初始化Bigquery SDK套件

In [1]:
import os
from google.cloud import bigquery

# 設置 Google Cloud 認證
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=r"/content/tibame-gad245-28-0305-1f39d5b14e0b.json"


# 初始化 BigQuery 客戶端
client = bigquery.Client()
print('connection done')


connection done


In [2]:
# 定義資料集參數
dataset_id = 'tibame_gad245_28_0312_dataset_us'  # 替換為想要建立的資料集名稱
project_id = 'tibame-gad245-28-0305'  # 替換為 Google Cloud 專案 ID
dataset_ref = client.dataset(dataset_id)

# 設定資料集的描述和其他選項
dataset = bigquery.Dataset(dataset_ref)
dataset.description = 'This is a new dataset created from Python.'
dataset.location = 'US'  # 設定資料集的地區位置

# 建立資料集
dataset = client.create_dataset(dataset, timeout=30)  # 可設定超時時間
print(f"dataset: {dataset.dataset_id} ,created")


dataset: tibame_gad245_28_0312_dataset_us ,created


In [3]:
# 定義資料集和資料表
table_id = 'age_table'
schema = [
    bigquery.SchemaField("name", "STRING"),
    bigquery.SchemaField("age", "INTEGER"),
]

# 定義表格參數
table_ref = client.dataset(dataset_id).table(table_id)
table = bigquery.Table(table_ref, schema=schema)

# 建立資料表
table = client.create_table(table)
print(f"table: {table.table_id} ,created")


table: age_table ,created


In [5]:

# 定義資料
rows_to_insert = [
    {"name": "Alice", "age": 25},
    {"name": "Bob", "age": 35},
    {"name": "Charlie", "age": 45},
]

# 插入資料
errors = client.insert_rows_json(f"{dataset_id}.{table_id}", rows_to_insert)
# insert:只會加入資料

if errors == []:
    print("data insert successfully")
else:
    print(f"error: {errors}")



data insert successfully


In [6]:
# 定義查詢語句
query = """
    SELECT name, age
    FROM `tibame_gad245_28_0312_dataset_us.age_table`
    WHERE age > 30
"""

# 執行查詢
query_job = client.query(query)

# 獲取結果
results = query_job.result()

# 印出結果
for row in results:
    print(f"name: {row.name}, age: {row.age}")



name: Bob, age: 35
name: Charlie, age: 45
name: Bob, age: 35
name: Charlie, age: 45


In [7]:

# 定義資料表參數
table_ref = client.dataset(dataset_id).table(table_id)

# 刪除資料表
client.delete_table(table_ref)
print(f"table: {table_id} ,deleted")


table: age_table ,deleted


In [8]:
# 定義 SQL 查詢以創建模型
query = """
CREATE OR REPLACE MODEL `tibame-gad245-28-0305.tibame_gad245_28_0312_dataset_us.imported_tf_model`
OPTIONS (MODEL_TYPE='TENSORFLOW',
         MODEL_PATH='gs://cloud-training-demos/txtclass/export/exporter/1549825580/*')
"""

# 執行 SQL 查詢以創建模型
query_job = client.query(query)
query_job.result()  # 等待查詢完成

print("TensorFlow model import BigQuery ML")


TensorFlow model import BigQuery ML


In [10]:
# 定義 SQL 查詢以使用模型進行預測，只取五筆非 NULL 的資料
query = """
SELECT *
FROM ML.PREDICT(MODEL `tibame-gad245-28-0305.tibame_gad245_28_0312_dataset_us.imported_tf_model`,
  (SELECT title AS input
   FROM `bigquery-public-data.hacker_news.full`
   WHERE title IS NOT NULL
   LIMIT 5)  -- 限制結果為五筆資料
)
"""

# 執行 SQL 查詢以進行預測
query_job = client.query(query)
results = query_job.result()  # 等待查詢完成

# 印出預測結果
for row in results:
    print(row)


Row(([0.8007680773735046, 0.10563275963068008, 0.09359916299581528], 'Structured (YC S23) Is Hiring'), {'dense_1': 0, 'input': 1})
Row(([0.8007680773735046, 0.10563275963068008, 0.09359916299581528], 'Curo (YC F24) Is Hiring'), {'dense_1': 0, 'input': 1})
Row(([0.8007680773735046, 0.10563275963068008, 0.09359916299581528], 'SciPhi (YC W24) Is Hiring'), {'dense_1': 0, 'input': 1})
Row(([0.0003840135468635708, 0.0009624646045267582, 0.9986535310745239], 'Design how business communicates. SendHub (YC W12) is looking for a Designer'), {'dense_1': 0, 'input': 1})
Row(([0.0932588204741478, 0.1606874316930771, 0.7460538148880005], 'Infracost (YC W21) Is Hiring Senior Software Engineers (GMT+2 to GMT-6)'), {'dense_1': 0, 'input': 1})


In [11]:

# 定義資料集參數
dataset_ref = client.dataset(dataset_id)

# 刪除資料集
client.delete_dataset(dataset_ref, delete_contents=True)
print(f"dataset: {dataset_id} ,deleted")


dataset: tibame_gad245_28_0312_dataset_us ,deleted
